# HuggingFace Datasets and Tigris

In order to get started with HuggingFace Datasets Tigris in an iPython notebook, you need the following:

1. A [Tigris account](https://storage.new)
1. A Tigris bucket in your account
1. An [access keypair](https://storage.new/accesskey) with Editor permissions for your bucket
1. A computer running Python 3.10 that has internet access (OS and CPU architecture does not matter)
1. The uv python environment manager

If you us [VS Code](https://code.visualstudio.com/) and have the [Development Containers](https://code.visualstudio.com/docs/devcontainers/containers) extension installed, clone this repository to your machine and run the command `Dev Containers: Reopen in Container`. This will automatically set up all of the dependencies you need to get started.

Install all of the dependencies for this project with `uv`:

In [1]:
! uv python install 3.10
! uv venv
! uv sync

Using CPython 3.10.18
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 75 packages in 0.70ms
Installed 66 packages in 2.08s                              
 + aiobotocore==2.23.0
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.13
 + aioitertools==0.12.0
 + aiosignal==1.3.2
 + asttokens==3.0.0
 + async-timeout==5.0.1
 + attrs==25.3.0
 + botocore==1.38.27
 + certifi==2025.6.15
 + charset-normalizer==3.4.2
 + comm==0.2.2
 + datasets==3.6.0
 + debugpy==1.8.14
 + decorator==5.2.1
 + dill==0.3.8
 + exceptiongroup==1.3.0
 + executing==2.2.0
 + filelock==3.18.0
 + frozenlist==1.7.0
 + fsspec==2025.3.0
 + hf-xet==1.1.5
 + huggingface-hub==0.33.2
 + idna==3.10
 + ipykernel==6.29.5
 + ipython==8.37.0
 + jedi==0.19.2
 + jmespath==1.0.1
 + jupyter-client==8.6.3
 + jupyter-core==5.8.1
 + matplotlib-inline==0.1.7
 + multidict==6.6.3
 + multiprocess==0.70.16
 + nest-asyncio==1.6.0
 + numpy==2.2.6
 + packaging==25.0
 + pandas==2.3.0
 + parso==0.8.4
 + pexpect==4.9.0
 +

Copy the contents of `.env.example` into `.env` and open `.env` in your editor.

If you are not using VS Code for this, you will need to open `.env` in your editor manually.

In [ ]:
! cp .env.example .env
! code .env || echo "open .env in your editor"

Put your access key in the `AWS_ACCESS_KEY_ID` field and put your secret key in the `AWS_SECRET_ACCESS_KEY` field.

For example, if your access key is `tid_NotKbzPHpJuoX` and your secret access key is `tsec_r++Q9iocfdf7Th`:

```patch
 ## Tigris configuration

 # Change these based on the access key you got from the web console
-AWS_ACCESS_KEY_ID=tid_access_key_id
-AWS_SECRET_ACCESS_KEY=tsec_secret_access_key
+AWS_ACCESS_KEY_ID=tid_NotKbzPHpJuoX
+AWS_SECRET_ACCESS_KEY=tsec_r++Q9iocfdf7Th
```

Then load the `.env` file into your notebook's environment:

In [ ]:
from dotenv import load_dotenv


load_dotenv()

True

Then make sure you got everything:

In [3]:
import os

dotenv_errs = []
for key in [
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_ENDPOINT_URL_S3",
    "AWS_ENDPOINT_URL_IAM",
    "AWS_REGION",
]:
    assert os.getenv(key) is not None, f"Environment variable {key} is not defined, please define it in .env"

Then set up the storage options for the datasets library:

In [4]:
storage_options = {
    "key": os.getenv("AWS_ACCESS_KEY_ID"),
    "secret": os.getenv("AWS_SECRET_ACCESS_KEY"),
    "endpoint_url": os.getenv("AWS_ENDPOINT_URL_S3"),
}

Make sure you have permissions to write files to your bucket:

In [5]:
import s3fs


# Change me!
bucket_name = "xe-datasets-demo"


fs = s3fs.S3FileSystem(**storage_options)
fs.write_text(f"/{bucket_name}/test.txt", "this is a test")
fs.rm(f"/{bucket_name}/test.txt")

[]

Load a dataset such as [mlabonne/FineTome-100k](http://hf.co/datasets/mlabonne/FineTome-100k):

In [6]:
from datasets import load_dataset
from IPython.display import display


dataset_name = "mlabonne/FineTome-100k"


dataset = load_dataset(dataset_name, split="train")
display(dataset)

/workspaces/huggingface-datasets-with-tigris/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

Then copy it to your bucket:

In [7]:
dataset.save_to_disk(
    f"s3://{bucket_name}/datasets/{dataset_name}",
    storage_options=storage_options,
)

Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:09<00:00, 10232.99 examples/s]


Then you can import it from Tigris in another workflow:

In [ ]:
from datasets import load_from_disk


dataset = load_from_disk(f"s3://{bucket_name}/datasets/{dataset_name}", storage_options=storage_options)


def remove_blue(row):
    """
    You can do any filtering or transformation here. This example transformation
    removes any conversations that mention the color "blue" so you can understand
    how to do advanced filtering or processing.
    """
    assert row['conversations'] is not None
    for conv in row['conversations']:
        assert conv['value'] is not None
        if "blue" in conv['value']:
            return False # remove the row

    return True # leave the row in


filtered_ds = dataset.filter(remove_blue)
display(filtered_ds)

Filter: 100%|██████████| 100000/100000 [00:01<00:00, 81854.37 examples/s]


Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 98035
})